[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pooya-mohammadi/Face/blob/master/_02_mtcnn_tf1/deep_utils_mtcnn.ipynb)

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
#! pip install datasets transformers seqeval

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/ai/.huggingface/token


Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
# !sudo apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.18.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](https://github.com/huggingface/notebooks/blob/main/examples/images/token_classification.png?raw=1)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [5]:
import transformers
from datasets import load_metric
from armanpers_dataset import HFArmanPersDataset
from datasets import ClassLabel, Sequence
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

In [6]:
model_checkpoint = "HooshvareLab/roberta-fa-zwnj-base"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [7]:
datasets = HFArmanPersDataset().dataset

Reusing dataset arman_pers (/home/ai/.pooya-mohammadi/arman_pers/ArmanPers/1.0.0)


Cache directory:  /home/ai/.pooya-mohammadi
Cache1 directory:  /home/ai/.pooya-mohammadi/arman_pers/ArmanPers/1.0.0


  0%|          | 0/3 [00:00<?, ?it/s]

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5122
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2561
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2562
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [8]:
datasets["train"][0]

{'id': '0',
 'tokens': ['به',
  'عنوان',
  'مثال',
  'وقتی',
  'نشریات',
  'مدافع',
  'اصول',
  'و',
  'ارزشها',
  'و',
  'منادی',
  'انقلاب',
  'و',
  'اسلام',
  'در',
  'بالاترین',
  'درجه',
  '،',
  'اولین',
  'و',
  'درشت\u200cترین',
  'تیتر',
  'نشریه',
  'خود',
  'را',
  'در',
  'صدر',
  'صفحه',
  'نخستین',
  '،',
  'به',
  'تکذیب',
  'اظهارات',
  'و',
  'نظریات',
  'مشاور',
  'رئیس\u200cجمهور',
  'با',
  'همین',
  'ترکیب',
  'عبارتی',
  'و',
  'البته',
  'از',
  'قول',
  'دیگران',
  'اختصاص',
  'می\u200cدهند',
  '،',
  'آیا',
  'در',
  'موارد',
  'مشابه',
  'نیز',
  'هر',
  'گاه',
  'خبر',
  'تکذیب',
  'متوجه',
  'و',
  'معطوف',
  'به',
  'شخصی',
  'باشد',
  'كه',
  'در',
  'زمره',
  'مشاوران',
  'يك',
  'مقام',
  'بلندمرتبه\u200cی',
  'دیگر',
  'است',
  '،',
  'خبر',
  'را',
  'عینا',
  'به',
  'همین',
  'درشتی',
  'و',
  'با',
  'همین',
  'ترکیب',
  'عبارتی',
  'در',
  'صدر',
  'صفحه',
  'نخست',
  'به',
  'چاپ',
  'می\u200cرساند',
  'و',
  'در',
  'آن',
  'مورد',
  'هم',
  'به

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [9]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=13, names=['B-event', 'B-fac', 'B-loc', 'B-org', 'B-pers', 'B-pro', 'I-event', 'I-fac', 'I-loc', 'I-org', 'I-pers', 'I-pro', 'O'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:

- "B-event": "رویداد",
- "B-fac": "امکانات",
- "B-loc": "موقعیت",
- "B-org": "سازمان",
- "B-pers": "شخص",
- "B-pro": "محصول",
- "I-event": "رویداد",
- "I-fac": "امکانات",
- "I-loc": "موقعیت",
- "I-org": "سازمان",
- "I-pers": "شخص",
- "I-pro": "محصول",
- "O": None

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [10]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['B-event',
 'B-fac',
 'B-loc',
 'B-org',
 'B-pers',
 'B-pro',
 'I-event',
 'I-fac',
 'I-loc',
 'I-org',
 'I-pers',
 'I-pro',
 'O']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [11]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,3130,"[مثلا, می‌توانیم, يك, سینی, پر, از, شن, را, در, گوشه‌ای, از, تراس, آپارتمان, يا, حیاط, خانه, قرار, دهیم, تا, کودکمان, اوقاتی, از, روز, را, به, بازی, با, شن, بگذراند, يا, جائی, را, فراهم, کنیم, كه, کودک, ما, با, آب, بازی, کند, و, در, آب, دست‌وپا, بزند, و, يا, شاید, محیط, آشپزخانه, برای, کودک, ما, جذاب, باشد, ،, در, این, صورت, می‌توانیم, مکانهائی, را, كه, در, دسترس, طفلمان, است, ،, از, وسائل, و, ظروف, بی‌خطر, پر, کنیم, ،, و, بگذاریم, کودکمان, اوقاتی, را, نیز, در, آشپزخانه, به, بازی, بپردازد, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,4137,"[آقای, اصلانی, از, دیگر, طرحهای, اداره, كل, نوسازی, استان, همدان, را, كه, در, سطح, کشور, برای, اولین, بار, اجرا, شده, است, ،, طرح, پاکسازی, و, جمع‌آوری, وسائل, اسقاطی, مدارس, خواند, و, گفت, :, در, اجرای, این, طرح, تاکنون, 0, کلاس, درسی, تخلیه, و, 0, مترمربع, فضای, آموزشی, آزاد, و, همچنین, 0, میز, و, 0, صندلی, بازسازی, شده, ست, .]","[O, B-pers, O, O, O, B-org, I-org, I-org, I-org, I-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,4421,"[ولی, فقیه, از, یک, سو, مجری, احکام, اسلام, است, و, از, سوی, دیگر, با, مجالس, مختلف, ـ, مجلس, شورای, اسلامی, ،, شورای, نگهبان, ،, مجمع, تشخیص, مصلحت, ـ, مشورت, می‌کند, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-org, I-org, I-org, O, B-org, I-org, O, B-org, I-org, I-org, O, O, O, O]"
3,4414,"[مقصود, از, اطلاق, ،, توسعه, و, گسترش, دائره, نفوذ, فقیه, است, که, این, نیز, لازمه, رهبری, و, اداره, یک, جامعه, است, و, رهبری, برای, تحقق, مصالح, امت, به, این, امر, نیاز, دارد, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,2983,"[خبرنگاران, خارجی, و, بویژه, آمریکائی, فعال, در, سازمان, ملل, كه, از, حضور, آلبرایت, در, این, اجلاس, تعجب, کرده, بودند, ،, اقدام, وي, را, نشان, دیگری, مبنی, بر, تلاش, آمریکا, جهت, نزدیک, شدن, به, ایران, قلمداد, می‌کنند, .]","[O, O, O, O, B-org, O, O, B-org, I-org, O, O, O, B-pers, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-org, O, O, O, O, B-org, O, O, O]"
5,1367,"[او, در, چند, فیلم, سینمائی, مثل, کامیت, منتز, هم, بازی, کرده, بود, .]","[O, O, O, O, O, O, B-pro, I-pro, O, O, O, O, O]"
6,2320,"[وزیر, اطلاعات, از, قدرتهای, نظامی, ،, اقتصادی, ،, علمی, ،, دیپلماسی, و, فرهنگ, ملی, به, عنوان, مؤلفه‌های, امنیت, ملی, کشور, یاد, کرد, و, گفت, :, همه, آنها, باید, در, اوج, اقتدار, و, هماهنگ, عمل, کنند, تا, کشور, به, يك, امنیت, پایدار, و, توسعه, كه, هدف, دولت, خدمتگزار, است, ،, دست, یابد, .]","[O, B-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,4983,"[0, ـ, حاضران, در, این, نشست, همچنان, معتقدند, ،, این, مجموعه, به, عنوان, نخستین, مخاطبان, طرح, جدید, واگذاری, اداره, کتابخانه‌های, عمومی, به, شوراهای, اسلامی, شهر, و, روستا, ،, توان, آنرا, دارند, تا, با, ارزیابی, کارشناسانه, موضوع, ،, متولیان, امر, را, در, تدوین, اصولی, و, ماندگار, چهارچوب, کار, ،, یاری, دهند, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-org, I-org, I-org, I-org, I-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,4567,"[وزیر, بهداشت, و, درمان, سپس, با, اشاره, به, نبود, بودجه, کافی, برای, درمان, بیماریهای, خاص, و, صعب‌العلاج, ،, از, مجلس, شورای, اسلامی, درخواست, کرد, که, در, بودجه, سال, آینده, مبلغ, قابل, توجهی, برای, کمک, به, درمان, این, گونه, بیماران, اختصاص, دهد, .]","[O, B-org, I-org, I-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-org, I-org, I-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,3475,"[رود, ارس, ،, دشت, مغان, ،, جمهوری, آذربایجان, ،, کوههای, تالش, در, استان, گیلان, ،, استان, زنجان, و, استان, آذربایجان, غربی, از, همسایگان, اردبیل, هستند, .]","[B-loc, 

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/302 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/855k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [14]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [15]:
print('sample: ', tokenizer("سلام این یک جمله است!"))

sample:  {'input_ids': [2, 3372, 1930, 1961, 2675, 1933, 110, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [16]:
print("tokenized sample: ",
      tokenizer(["سلام", ",", "این", "یک", "جمله", "تکه", "شده", "به", "کلمات", "است", "."], is_split_into_words=True))

tokenized sample:  {'input_ids': [2, 3372, 119, 1930, 1961, 2675, 6600, 1983, 1923, 5525, 1933, 121, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [17]:
example = datasets["train"][4]
print(example["tokens"])

['حمید', 'طاهایی', 'افزود', ':', 'برای', 'اجرای', 'این', 'طرحها', '0', 'میلیارد', 'و', '0', 'میلیون', 'ریال', 'اعتبار', 'هزینه', 'شده', 'است', '.']


In [18]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'حمید', 'طاه', '##ایی', 'افزود', ':', 'برای', 'اجرای', 'این', 'طرحها', '0', 'میلیارد', 'و', '0', 'میلیون', 'ریال', 'اعتبار', 'هزینه', 'شده', 'است', '.', '[SEP]']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can use a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [20]:
len(example[f"ner_tags"]), len(tokenized_input["input_ids"])

(19, 22)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [21]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [23]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

22 22


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [24]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [27]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [28]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[2, 1923, 2179, 3065, 2849, 11450, 7969, 5281, 623, 3019, 2001, 623, 9436, 2279, 4301, 623, 2814, 1921, 6051, 3687, 590, 2613, 623, 10846, 9, 2287, 10791, 6936, 1969, 1937, 1921, 6482, 2730, 3279, 590, 1923, 10084, 8736, 623, 14930, 4136, 1, 9, 4937, 1927, 2301, 3643, 8502, 623, 2486, 1925, 5959, 3859, 3425, 1924, 9, 2493, 590, 595, 2087, 1921, 2891, 3339, 2058, 2063, 4814, 2549, 10084, 3828, 623, 10729, 1923, 3325, 2094, 1, 1921, 11925, 10145, 1, 3091, 41732, 9, 682, 2102, 1933, 590, 2549, 1937, 22781, 1923, 2301, 29767, 623, 1927, 2301, 3643, 8502, 1921, 6482, 2730, 2753, 1923, 4042, 1924, 9, 4639, 623, 1921, 595, 1125, 2133, 1951, 1923, 2331, 3998, 2118, 625, 1110, 6628, 595, 1125, 2576, 1921, 2483, 623, 5049, 9, 682, 2723, 4826, 1934, 2126, 2179, 4136, 10002, 1, 41732, 1, 1924, 9, 2043, 593, 2154, 6936, 2690, 2726, 2006, 590, 12004, 2133, 5358, 595, 2564, 2082, 2204, 9, 2782, 1, 2187, 2549, 1937, 7791, 2074, 623, 2532, 2690, 1937, 2807, 2082, 2218, 1933, 593, 7585, 9

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [29]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [14]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/roberta-fa-zwnj-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at HooshvareLab/roberta-fa-zwnj-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [34]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [35]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [36]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [37]:
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'pers': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [40]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5122
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 963


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.131388,0.637114,0.653521,0.645213,0.961124
2,0.187300,0.114588,0.694591,0.712228,0.703299,0.966354
3,0.187300,0.113716,0.710990,0.726994,0.718903,0.967641


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2561
  Batch size = 16
Saving model checkpoint to distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500
Configuration saved in distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500/config.json
Model weights saved in distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_

TrainOutput(global_step=963, training_loss=0.13268381437656168, metrics={'train_runtime': 1069.9042, 'train_samples_per_second': 14.362, 'train_steps_per_second': 0.9, 'total_flos': 366778420203240.0, 'train_loss': 0.13268381437656168, 'epoch': 3.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [41]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2561
  Batch size = 16


{'eval_loss': 0.11371581256389618,
 'eval_precision': 0.7109897851356112,
 'eval_recall': 0.7269944174320188,
 'eval_f1': 0.7189030362389814,
 'eval_accuracy': 0.9676411677407972,
 'eval_runtime': 43.311,
 'eval_samples_per_second': 59.13,
 'eval_steps_per_second': 3.717,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [42]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2561
  Batch size = 16


{'event': {'precision': 0.4327731092436975,
  'recall': 0.4724770642201835,
  'f1': 0.4517543859649123,
  'number': 218},
 'fac': {'precision': 0.5536723163841808,
  'recall': 0.5212765957446809,
  'f1': 0.5369863013698629,
  'number': 188},
 'loc': {'precision': 0.7524154589371981,
  'recall': 0.7971849008317339,
  'f1': 0.7741534638086364,
  'number': 1563},
 'org': {'precision': 0.6716500553709857,
  'recall': 0.6935391652372784,
  'f1': 0.6824191279887483,
  'number': 1749},
 'pers': {'precision': 0.792191435768262,
  'recall': 0.851150202976996,
  'f1': 0.8206131767775603,
  'number': 1478},
 'pro': {'precision': 0.5586854460093896,
  'recall': 0.3333333333333333,
  'f1': 0.4175438596491227,
  'number': 357},
 'overall_precision': 0.7109897851356112,
 'overall_recall': 0.7269944174320188,
 'overall_f1': 0.7189030362389814,
 'overall_accuracy': 0.9676411677407972}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```